In [ ]:
from PIL import Image

from resizeimage import resizeimage


with open('images/5a683f4e2d00004900942a0f.jpeg', 'r+b') as f:
    with Image.open(f) as image:
        cover = resizeimage.resize_cover(image, [200, 100])
        cover.save('test-image-cover.jpeg', image.format)

In [1]:
import json
with open('train_.txt') as json_file:  
    train_data = json.load(json_file)
    
I=[]
for i in train_data:
    if i['image']=='':
        pass
    else:
        I.append([i['image'],i['category']])

In [2]:
a=[]
for i in train_data:
    a.append(i['category'])
b=set(a)

In [3]:
d = {}
for k,v in enumerate(b):
    d[v] = k


In [ ]:
len(I)

In [4]:
I

[['58daa1191d0000cf3b7d0f72.png', 'BLACK VOICES'],
 ['578207511b00001900f6d04c.jpg', 'HEALTHY LIVING'],
 ['590c9e911400001f00f8b6ce.jpeg', 'HEALTHY LIVING'],
 ['5a0340731b00001500018f80.jpg', 'WORLD NEWS'],
 ['59dc80532d000097173097cf.png', 'BLACK VOICES'],
 ['5a6438811c00002600811b42.png', 'COMEDY'],
 ['59e85c36150000860a746603.jpeg', 'WORLD NEWS'],
 ['595160b21700006f07102dc6.jpg', 'HEALTHY LIVING'],
 ['o-BARACK-OBAMA-facebook.jpg', 'BLACK VOICES'],
 ['5a991af41e000008087ad42b.png', 'COMEDY'],
 ['5a021b081400002500d7f524.jpg', 'WORLD NEWS'],
 ['5a155da7150000b534859682.jpeg', 'WORLD NEWS'],
 ['5840fe911700002500e7d66e.jpeg', 'BLACK VOICES'],
 ['561cfb4b1400002b003c8102.jpeg', 'SPORTS'],
 ['569178031a00002d00594311.png', 'BLACK VOICES'],
 ['583478071700002500e7bdbe.png', 'COMEDY'],
 ['5a62f6d41f00009a00db9892.png', 'COMEDY'],
 ['581526a4190000df02c2fceb.jpg', 'HEALTHY LIVING'],
 ['57ad6698180000ad02bca5b3.jpeg', 'SPORTS'],
 ['59a808b217000020002879ee.jpeg', 'WORLD NEWS'],
 ['5afd81fd1

In [ ]:
for i in I:
    with open('images/'+i[0], 'r+b') as f:
        with Image.open(f) as image:
            cover = resizeimage.resize_cover(image, [200, 100])
            cover.save('resized_images/'+ i[0], image.format)    

In [5]:
import numpy as np
from PIL import Image
import matplotlib.image as img
Im=[]
L=[]

for i in I:
    try:
        image = img.imread("resize_images_64/" + i[0])
        Im.append(image)
        L.append(d[i[1]])
        
    except:
        img = Image.open('resize_images_64/' + i[0]).convert('RGBA')
        arr = np.array(img)
        Im.append(arr)
        L.append(d[i[1]])

In [6]:
img_=np.array(Im)
L_=np.asarray(L)

In [7]:
train_data=img_
train_label=L_

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.05)

In [9]:
X_train_mean = X_train - np.mean(X_train,axis=(0,1,2))#subracting the mean of channel by using 'axis'
X_test_mean =X_test - np.mean(X_train,axis=(0,1,2))

In [10]:
X_train_meanstd = X_train_mean/np.std(X_train,axis=(0,1,2))#We are making standart normalization
X_test_meanstd = X_test_mean/np.std(X_train,axis=(0,1,2))

In [11]:
X_train_norm = X_train_meanstd
X_test_norm = X_test_meanstd

In [19]:
X_train_norm.shape

(10558, 64, 64, 4)

In [12]:
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K

C:\Users\Adil\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
from keras.layers import MaxPooling2D, BatchNormalization, Dropout
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import plot_model

x = Input(shape=(64, 64, 4))#I have just write the right functions and layers as given above from Keras documentation
c1 = Conv2D(32, (6, 6), strides=(1, 1),padding='same')(x)
b1 = BatchNormalization()(c1)
a1 = Activation('relu')(b1)
c2 = Conv2D(32, (3, 3), strides=(1, 1),padding='valid')(a1)
b2 = BatchNormalization()(c2)
a2 = Activation('relu')(b2) 
p2 = MaxPooling2D(pool_size=(2, 2))(a2)
d2 = Dropout(0.25)(p2)
f2 = Flatten()(d2)
h3 = Dense(100)(f2)
b3 = BatchNormalization()(h3)
a3 = Activation('relu')(b3)
d3 = Dropout(0.5)(a3)
z = Dense(5)(d3)
p = Activation('softmax')(z)

model = Model(inputs=x, outputs=p)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.summary()
plot_model(model, to_file="model_Conv.png", show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
batch_normalization_4 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
activation_54 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 62, 62, 32)        9248      
_________________________________________________________________
batch_normalization_5 (Batch (None, 62, 62, 32)        128       
_________________________________________________________________
activation_55 (Activation)   (None, 62, 62, 32)        0         
__________

In [14]:
from keras.utils import to_categorical
history = model.fit(X_train_norm, batch_size=64, y=to_categorical(y_train), verbose=1, validation_split=0.10, 
          shuffle=False, epochs=5)

Instructions for updating:
Use tf.cast instead.
Train on 9502 samples, validate on 1056 samples
Epoch 1/5
9502/9502 [==============================] - 273s 29ms/step - loss: 1.7392 - acc: 0.3189 - val_loss: 1.5186 - val_acc: 0.3949
Epoch 2/5
9502/9502 [==============================] - 261s 28ms/step - loss: 1.5216 - acc: 0.3791 - val_loss: 1.4494 - val_acc: 0.4006
Epoch 3/5
9502/9502 [==============================] - 259s 27ms/step - loss: 1.4015 - acc: 0.4299 - val_loss: 1.4845 - val_acc: 0.3826
Epoch 4/5
9502/9502 [==============================] - 259s 27ms/step - loss: 1.2971 - acc: 0.4770 - val_loss: 1.4657 - val_acc: 0.3712
Epoch 5/5
9502/9502 [==============================] - 262s 28ms/step - loss: 1.1892 - acc: 0.5355 - val_loss: 1.5449 - val_acc: 0.3570
